In [ ]:
import numpy as np
import glob
import os
import pylab
import scipy.misc
import skimage
from skimage import io
from skimage import transform
from scipy import ndimage
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
def get_padded_slices(slices, pad, label):
    
    new_slices = []
    for i,s in enumerate(slices):
            
        start = s.start
        stop = s.stop
                
        if i == 3:
            
            # my edits to prevent slicing of the channels
            new_start = 0
            new_stop = 3
            # end of my edits
            
            #fairly's original code below
            #new_slices.append(s)
            
            # my edited code
            new_slices.append(slice(new_start, new_stop))
            continue
            
        if start-pad < 0:
            new_start = 0
        else:
            new_start = start-pad
            
        if stop+pad > label.shape[i]:
            new_stop = label.shape[i]
        else:
            new_stop = stop+pad
            
        new_slices.append(slice(new_start, new_stop))
        
    return tuple(new_slices)

In [ ]:
def label_mask(mask):
    
    label, n = ndimage.label(mask)
    return label, n

In [ ]:
def load_mask(mask_file):
    
    mask = io.imread(mask_file)
    return mask

In [ ]:
def load_original_image(image_file):
    
    original_image = io.imread(image_file)
    return original_image

In [ ]:
def crop_image(original_image, new_slices):
    
    cropped_image = original_image[new_slices]
    return cropped_image

In [ ]:
def save_cropped_image(image_file):
    
    skimage.io.imsave((image_file + str(i)))

In [ ]:
def get_slices(label):
    
    slices = ndimage.find_objects(label)
    return slices

In [ ]:
def extract_inclusions(mask_file, image_file, pad):
    
    mask = load_mask(mask_file)
    original_image = load_original_image(image_file)
    label, n = label_mask(mask)
    slices = get_slices(label)
    
    for i in range (0, n):
        
        new_slices = get_padded_slices(slices[i], pad, label)
        cropped_image = original_image[new_slices]
        skimage.io.imsave((image_file[:-4] + ' cropped ' + str(i) + '.tif'), cropped_image)
        
        
        
        

In [ ]:
def get_64x64x64_cubes(slices, label):
    
    new_slices = []
    for i,s in enumerate(slices):
            
        start = s.start
        stop = s.stop
                
        if i == 3:
            
            # my edits to prevent slicing of the channels
            new_start = 0
            new_stop = 3
            # end of my edits
            
            #fairly's original code below
            #new_slices.append(s)
            
            # my edited code
            new_slices.append(slice(new_start, new_stop))
            continue
            
        if round((start + stop)/2)-32 < 0:
            break
        else:
            new_start = round((start + stop)/2) - 32
            
        if round((start + stop)/2)+32 > label.shape[i]:
            break
        else:
            new_stop = round((start + stop)/2)+32
            
        new_slices.append(slice(new_start, new_stop))
        
    return tuple(new_slices)

In [ ]:
def extract_cubed_inclusions(mask_file, image_file):
    
    mask = load_mask(mask_file)
    original_image = load_original_image(image_file)
    label, n = label_mask(mask)
    slices = get_slices(label)
    counter = 0
    for i in range (0, n):
        
        new_slices = get_64x64x64_cubes(slices[i], label)
        
        if len(new_slices) < 4:
            continue
        
        else:
            cropped_image = original_image[new_slices]
            skimage.io.imsave((image_file[:-4] + ' cropped ' + str(counter) + '.tif'), cropped_image)
            counter+=1
    return slices, counter

In [ ]:
def get_padded_slices_v2(slices, pad, label):
    
    new_slices = []
    for i,s in enumerate(slices):
            
        start = s.start
        stop = s.stop
                
        if i == 3:
            
            # my edits to prevent slicing of the channels
            new_start = 0
            new_stop = 3
            # end of my edits
            
            #fairly's original code below
            #new_slices.append(s)
            
            # my edited code
            new_slices.append(slice(new_start, new_stop))
            continue
            
        if start-pad < 0:
            new_start = 0
        else:
            new_start = start-pad
            
        if stop+pad > label.shape[i]:
            new_stop = label.shape[i]
        else:
            new_stop = stop+pad
        
        if new_stop-new_start > 64:
            break
        else:
            new_slices.append(slice(new_start, new_stop))
        
    return tuple(new_slices)

In [ ]:
def pad_to_64(cropped_shape, cropped_image):
    for i in range(0,3):
        if cropped_shape[i] % 2 == 1:
            if i == 0:
                a = int((64-cropped_shape[i] + 1)/2)
                b = a-1
            if i == 1:
                c = int((64-cropped_shape[i] + 1)/2)
                d = c-1
            if i == 2:
                e = int((64-cropped_shape[i] + 1)/2)
                f = e-1
        else:
            if i == 0:
                a = int((64-cropped_shape[i])/2)
                b = int((64-cropped_shape[i])/2)
            if i == 1:
                c = int((64-cropped_shape[i])/2)
                d = int((64-cropped_shape[i])/2)
            if i == 2:
                e = int((64-cropped_shape[i])/2)
                f = int((64-cropped_shape[i])/2)
                
    padded_cropped_image = np.pad(cropped_image,((a,b), (c,d), (e,f), (0, 0)), 'constant')
    return padded_cropped_image

In [ ]:
def extract_padded_cropped_inclusions(mask_file, image_file, pad):
    
    mask = load_mask(mask_file)
    original_image = load_original_image(image_file)
    label, n = label_mask(mask)
    slices = get_slices(label)
    counter = 0
    for i in range (0, n):
        
        new_slices = get_padded_slices_v2(slices[i], pad, label)
        
        if len(new_slices) < 4:
            continue
        
        else:
            cropped_image = original_image[new_slices]
            cropped_shape = cropped_image.shape
            padded_cropped_image = pad_to_64(cropped_shape, cropped_image)        
            skimage.io.imsave((image_file[:-4] + ' padded cropped ' + str(counter) + '.tif'), padded_cropped_image)
            counter+=1
    return slices, counter

In [ ]:
for i in range(0, len(file_listnobuff)):
    x = io.imread(file_listnobuff[i])
    y = np.max(x, axis=0)
    z = y[:, :, ::-1]
    z = skimage.exposure.rescale_intensity(z, in_range='uint12')
    z = skimage.img_as_ubyte(z)
    z = skimage.transform.resize(z, (512,512,3), order=3)
    z[:,:,2] = 0
    image_file = file_listnobuff[i]
    skimage.io.imsave(('zMIP ' + image_file[:-4] + '.tif'), z)

In [ ]:
for i in range(0, len(file_listnobuff)):
    x = io.imread(file_listnobuff[i])
    y = np.max(x, axis=1)
    z = y[:, :, ::-1]
    z = skimage.exposure.rescale_intensity(z, in_range='uint12')
    z = skimage.img_as_ubyte(z)
    z = skimage.transform.resize(z, (512,512,3), order=3)
    z[:,:,2] = 0
    image_file = file_listnobuff[i]
    skimage.io.imsave(('yMIP ' + image_file[:-4] + '.tif'), z)

In [ ]:
for i in range(0, len(file_listnobuff)):
    x = io.imread(file_listnobuff[i])
    y = np.max(x, axis=2)
    z = y[:, :, ::-1]
    z = skimage.exposure.rescale_intensity(z, in_range='uint12')
    z = skimage.img_as_ubyte(z)
    z = skimage.transform.resize(z, (512,512,3), order=3)
    z[:,:,2] = 0
    image_file = file_listnobuff[i]
    skimage.io.imsave(('xMIP ' + image_file[:-4] + '.tif'), z)

In [1]:
cd /home/monuki/Documents/code_repository

/home/monuki/Documents/code_repository


In [2]:
!ls

biondi_classifier_ai_code.ipynb		MIP annotation tool.ipynb
Biondi_inclusion_extraction_code.ipynb	Untitled1.ipynb
Brett_annotation_UCI-43-18.ipynb	Untitled2.ipynb
code_repository				Untitled.ipynb
Irene_annotation_UCI-43-18.ipynb	Zohaer_annotation_UCI-43-18.ipynb
MIP annotation tool (copy).ipynb


In [3]:
!git init

Reinitialized existing Git repository in /home/monuki/Documents/code_repository/.git/


In [ ]:
!git init add biondi_cl